In [1]:
import pandas as pd
import unidecode
from tqdm.notebook import tqdm

import torch

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments
from trl import DPOTrainer
from datasets import Dataset

from utils import remove_diacritics

model_name = "google/flan-t5-small"
LORA_RUN = False

model_output_dir = f"dpo_{model_name}" 
model_output_dir += "_lora" if LORA_RUN else ""
print(f"{model_output_dir=}")


model_output_dir='dpo_google/flan-t5-small'


In [2]:
df = pd.read_csv("data/t5-small.csv")

df['chosen'] = df['translations'].apply(remove_diacritics)
df["sentence"] = df["sentence"].apply(lambda x: f"translate English to Romanian: {x}")
df = df.rename(columns={"translations": "rejected", "sentence": "prompt"})
train_dataset = Dataset.from_dict({col: df[col].values.tolist() for col in df.columns})
df

,prompt,rejected,chosen
0,translate English to Romanian: I ate the cheese.,Am mâncat brânza.,Am mancat branza.
1,translate English to Romanian: Today is Monday.,Astăzi este ziua de luni.,Astazi este ziua de luni.
2,translate English to Romanian: Does he speak E...,Vorbeşte el limba engleză?,Vorbeste el limba engleza?
3,translate English to Romanian: I'm sort of tired.,Sunt oarecum obosit.,Sunt oarecum obosit.
4,translate English to Romanian: I am indebted t...,Sunt îndatorat acestuia.,Sunt indatorat acestuia.
...,...,...,...
15801,translate English to Romanian: It would be a d...,Ar fi o sarcină dificilă.,Ar fi o sarcina dificila.
15802,translate English to Romanian: I ate a burdock...,Am mâncat o tempura de rădăcini burdice.,Am mancat o tempura de radacini burdice.
15803,translate English to Romanian: You say you've ...,Aţi spus că aţi văzut o UFO?,Ati spus ca ati vazut o UFO?
15804,translate English to Romanian: It's a good sen...,"Oricum, este o frază bună.","Oricum, este o fraza buna."


In [3]:
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
ref_model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# # This is an experiment where the base layers are frozen

# for name, param in model.named_parameters():
#     if 'lm_head' not in name:
#         param.requires_grad = False

In [5]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-6,
    num_train_epochs=2, 
    logging_dir=f"{model_output_dir}/logs",  
    logging_steps=10
)

dpo_trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    beta=0.1,
    max_prompt_length=128,
    max_length=1536,
)


d:\Python310\lib\site-packages\trl\trainer\dpo_trainer.py:300: UserWarning: When using DPODataCollatorWithPadding with an encoder decoder architecture, you should set `max_target_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
d:\Python310\lib\site-packages\trl\trainer\dpo_trainer.py:316: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/15806 [00:00<?, ? examples/s]

In [6]:
dpo_trainer.train()

  0%|          | 0/988 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6716, 'learning_rate': 9.898785425101213e-07, 'rewards/chosen': -0.05384952947497368, 'rewards/rejected': -0.11240199953317642, 'rewards/accuracies': 0.596875011920929, 'rewards/margins': 0.05855246260762215, 'logps/rejected': -25.946918487548828, 'logps/chosen': -43.5444221496582, 'logits/rejected': -9.902381896972656, 'logits/chosen': -9.617629051208496, 'epoch': 0.02}
{'loss': 0.6814, 'learning_rate': 9.797570850202428e-07, 'rewards/chosen': -0.08459533751010895, 'rewards/rejected': -0.12115220725536346, 'rewards/accuracies': 0.5625, 'rewards/margins': 0.03655686974525452, 'logps/rejected': -24.522146224975586, 'logps/chosen': -40.167640686035156, 'logits/rejected': -9.997121810913086, 'logits/chosen': -9.800909042358398, 'epoch': 0.04}
{'loss': 0.6753, 'learning_rate': 9.696356275303643e-07, 'rewards/chosen': -0.045400477945804596, 'rewards/rejected': -0.09452062845230103, 'rewards/accuracies': 0.5718749761581421, 'rewards/margins': 0.04912015050649643, 'logps/rejected':

TrainOutput(global_step=988, training_loss=0.5481572221165244, metrics={'train_runtime': 344.2521, 'train_samples_per_second': 91.828, 'train_steps_per_second': 2.87, 'train_loss': 0.5481572221165244, 'epoch': 2.0})

In [9]:
def translate_batch(batch, model=model, tokenizer=tokenizer):
    inputTokens = tokenizer(batch, padding=True, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(input_ids = inputTokens['input_ids'].long(), attention_mask=inputTokens['attention_mask'], max_new_tokens=128)
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for i, j in zip(batch, outputs):
        print(f"prompt: <{i}>, output:{j}")


translate_batch(df.prompt.values.tolist()[:10])

['Am sa sa sa.', 'Astăzi este luni.', 'e sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa s', 'Sunt o sase de dră.', 'Am adobtat.', 'Acesta a refuzat să spune mai mult.', 'i ai, dle me.', 'Springa a sa sa sa sa sa sa.', 'Sprăta va acţiona în curând.', 'Nu trebuie să apţine în aprovizionarea.']
prompt: <translate English to Romanian: I ate the cheese.>, output:Am sa sa sa.
prompt: <translate English to Romanian: Today is Monday.>, output:Astăzi este luni.
prompt: <translate English to Romanian: Does he speak English?>, output:e sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa sa s
prompt: <translate English to Romanian: I'm sort of tired.>, output:Sunt o sase de dră.
prompt: <translate English to Romanian: I am indebted to him.>, output:Am adobtat.
prompt: <translate English to Romanian: He refused to say more about that.>, output:Acesta a refu